Author:
        
        PARK, JunHo, junho@ccnets.org

    COPYRIGHT (c) 2024. CCNets. All Rights reserved.

# Cheat Your Decision-Making Model: Manipulating Credit Card Data with Causal Inference

## Introduction

In this tutorial, we delve into the intricate world of causal inference to manipulate decision-making models in the domain of credit card fraud detection. Utilizing a Cooperative Network (CCNet), we explore how subtle manipulations in non-fraudulent data can expose and test the vulnerabilities of machine learning models. This demonstration not only highlights the robustness and weaknesses of current fraud detection systems but also illustrates the ethical considerations and potential risks inherent in data manipulation.


## Tutorial Goals

This tutorial provides a guide on using causal inference to manipulate data and evaluate its impact on machine learning models. We will cover:

1. **Training a Binary Classifier**: Train a model to distinguish between fraudulent and non-fraudulent transactions using real credit card data.
2. **Generating Manipulated Data with CCNet**: Use CCNet's Explainer, Reasoner, and Producer networks to create data that mimics non-fraudulent transactions but is designed to deceive the classifier by switching decision label factors.
3. **Evaluating Classifier Performance**: Test the classifier on both original and manipulated datasets to assess robustness. The classifier should perform well on both the original test set and the CCNet-recreated test set, indicating that CCNet has learned the data distribution that the classifier mapped for decision making.
4. **Comparative Dataset Analysis**:
   - **Testset A**: Real Test Data - Authentic credit card transactions used as a baseline to evaluate the classifier's performance.
   - **Testset B**: CCNet-generated data with random labels but maintaining the same frequency distribution as the original. The classifier's performance should be similar to Testset A, showing that CCNet understands the classifier's decision-making process.
   - **Testset C**: CCNet-generated data with original labels. With Testset B, it is proved that CCNet creates the genuine data distribution for any case of decision label. However, as most information of the data is in the explanation vector, it is easy to manipulate by changing the decision factor of the fraud data. In short, it is easy to manipulate the decision-making process by causal inference.
5. **Ethical Considerations**: Discuss the ethical implications of data manipulation in machine learning, emphasizing the need for robustness in model training and the risks of misuse in applications like fraud detection.


In [ ]:
import sys
path_append = "../"
sys.path.append(path_append)  # Go up one directory from where you are.


In [ ]:
# https://fraud-detection-handbook.github.io/fraud-detection-handbook/Chapter_7_DeepLearning/FeedForwardNeuralNetworks.html
import pandas as pd 

dataroot = path_append + "../data/credit_card_fraud_detection/creditcard.csv"
df = pd.read_csv(dataroot)
df

In [ ]:

print('No Frauds', round(df['Class'].value_counts()[0] / len(df) *100,2), '%of the dataset')
print('Frauds', round(df['Class'].value_counts()[1] / len(df) *100,2), '%of the dataset')

In [ ]:
import torch
from sklearn.preprocessing import StandardScaler, RobustScaler
from torch.utils.data import Dataset

# Initialize scalers
standard_sc = StandardScaler()
robust_sc = RobustScaler()
# Scale 'Amount' and 'Time' columns separately
df['scaled_amount'] = robust_sc.fit_transform(df['Amount'].values.reshape(-1, 1))
df['scaled_time'] = robust_sc.fit_transform(df['Time'].values.reshape(-1, 1))

# Drop original 'Time' and 'Amount' columns
df.drop(['Time', 'Amount'], axis=1, inplace=True)

# Move the 'Class' column to the end
class_column = df.pop('Class')
df = pd.concat([df, class_column], axis=1)

# Scale the rest of the features (excluding the class column)
df.iloc[:, :-1] = standard_sc.fit_transform(df.iloc[:, :-1])

# Calculate the number of features and classes
n_features = len(df.iloc[:, :-1].columns)
n_classes = len(df.iloc[:, -1:].columns)

print(n_features, n_classes)

In [ ]:
df.head()

In [ ]:
# Defining the labeled and unlabeled dataset classes
class LabeledDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        vals = torch.tensor(self.x[index], dtype=torch.float32)
        label = torch.tensor(self.y[index], dtype=torch.long)
        return vals, label

#### Dataset Splitting for Training and Testing

The original dataset is split into training and testing parts to evaluate the model's performance accurately. This step is crucial for validating the effectiveness of the training on unseen data.


In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the dataset into training and test sets for model evaluation
df_train, df_test = train_test_split(df, test_size=0.5, shuffle=True, random_state=42)
X_train, y_train = df_train.iloc[:, :-1].values, df_train.iloc[:, -1:].values
X_test, y_test = df_test.iloc[:, :-1].values, df_test.iloc[:, -1:].values

# Labeled datasets for supervised learning tasks
trainset = LabeledDataset(X_train, y_train)  # Corrected to include training data
testset = LabeledDataset(X_test, y_test)     # Test set with proper labels

# Printing the shapes of the datasets for verification
print(f"Labeled Trainset Shape: {len(trainset)}, {trainset.x.shape[1]}")
print(f"Labeled Testset Shape: {len(testset)}, {testset.x.shape[1]}")

#### Initial Setup and Model Configuration

This section initializes the environment by setting a fixed random seed to ensure reproducibility of results. It imports necessary configurations and initializes model parameters with specific configurations. The model specified here is set to have no core model but uses a 'tabnet' encoder model for data processing, which is particularly tailored for structured or tabular data like credit card transactions.


In [ ]:
# Set a fixed random seed for reproducibility of experiments
from nn.utils.init import set_random_seed
set_random_seed(0)

# Importing configuration setups for ML parameters and data
import torch
from tools.setting.ml_params import MLParameters
from tools.setting.data_config import DataConfig
from trainer_hub import TrainerHub

# Configuration for the data handling, defining dataset specifics and the task type
data_config = DataConfig(dataset_name='CreditCardFraudDetection', task_type='binary_classification', obs_shape=[n_features], label_size=n_classes, explain_size=n_features - n_classes)

# Initializing ML parameters without a core model and setting the encoder model to 'tabnet' with specific configurations
ml_params = MLParameters(core_model='tabnet', encoder_model='none')

# Setting training parameters and device configuration
ml_params.training.num_epoch = 10
ml_params.model.core_config.num_layers = 3
ml_params.algorithm.error_function = 'mse'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Create a TrainerHub instance to manage training and data processing
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=True, use_wandb=False)


In [ ]:
load_model = False
if load_model:
    trainer_hub.load_trainer(core_model = True)
else:
    trainer_hub.train(trainset)

In [ ]:
# DataLoader for processing training data in larger batches without shuffling
test_loader = torch.utils.data.DataLoader(dataset=testset, batch_size=256, shuffle=False, drop_last=False)

# CCNet setup for generating synthetic manipulated data
ccnet = trainer_hub.core_ccnet
manipulated_data = None
random_labels = None
original_labels = None
probability_positive = 0.0017 # 0.17% of the data is positive

# Generate synthetic data to enhance the diversity of the training dataset
for data, labels in test_loader:
    data = data.to(device)
    labels = labels.to(device)
    
    # Create random labels with 0.17% positive (1) and 99.83% negative (0) labels
    random_label = (torch.rand(labels.size(0)).to(device) < probability_positive).float()
    random_label = torch.nn.functional.one_hot(random_label.to(torch.int64), num_classes=2).float()

    # Use CCNet to explain the original data and generate synthetic counterparts
    explanations = ccnet.explain(data)
    synthetic_data = ccnet.produce(random_label, explanations)
    
    # Detach synthetic data and labels from GPU for further processing
    synthetic_data = synthetic_data.detach().cpu()
    random_label = random_label.detach().cpu().argmax(dim=-1).unsqueeze(-1)
    labels = labels.detach().cpu()

    # Accumulate the generated data for analysis and training
    if manipulated_data is None:
        manipulated_data = synthetic_data
        random_labels = random_label
        original_labels = labels
    else:
        manipulated_data = torch.cat([manipulated_data, synthetic_data], dim=0)
        random_labels = torch.cat([random_labels, random_label], dim=0)
        original_labels = torch.cat([original_labels, labels], dim=0)

# Output the shapes of the datasets for verification
print(f"Manipulated Data Shape: {manipulated_data.shape}")
print(f"Reversed Labels Shape: {random_labels.shape}")
print(f"Original Labels Shape: {original_labels.shape}")

# Create datasets for both synthetic and original label scenarios
random_label_testset = LabeledDataset(manipulated_data.numpy(), random_labels.numpy())
original_label_testset = LabeledDataset(manipulated_data.numpy(), original_labels.numpy())


#### Training Supervised Models

This section outlines the process of training supervised learning models using both original and synthetic datasets. The `train_supervised_model` function is designed to iterate through the dataset, perform forward passes, compute loss, and update model weights using backpropagation.


In [ ]:
def train_supervised_model(model, dataset, num_epoch=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    # Initialize the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # Ensure reproducibility by resetting the random seed
    # Create DataLoader for batch processing
    trainloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)
    # Training loop
    for epoch in range(num_epoch):  # Train for 2 epochs as an example
        for i, (data, label) in enumerate(trainloader):
            data = data.to(device).clone().detach()
            label = label.to(device).float()
            # Perform forward pass
            output = model(data)
            # Compute loss
            loss = torch.nn.functional.binary_cross_entropy(output, label)
            # Backward pass to compute gradients
            loss.backward()
            # Update weights
            optimizer.step()
            # Reset gradients
            optimizer.zero_grad()


In [ ]:

class MLP(torch.nn.Module):
    def __init__(self, input_size, output_size, num_layers=4, hidden_size=256):
        super(MLP, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        # Create a list to hold all layers
        layers = []
        
        # Input layer
        layers.append(torch.nn.Linear(input_size, hidden_size))
        layers.append(torch.nn.ReLU())
        
        # Hidden layers
        for _ in range(num_layers - 2):
            layers.append(torch.nn.Linear(hidden_size, hidden_size))
            layers.append(torch.nn.ReLU())
        
        # Output layer
        layers.append(torch.nn.Linear(hidden_size, output_size))
        
        # Register all layers
        self.layers = torch.nn.Sequential(*layers)

    def forward(self, x):
        x = self.layers(x)
        return torch.sigmoid(x)
    
# Initialize and train a model on the recreated dataset
decision_making_model = MLP(input_size=n_features, output_size=n_classes).to(device)
train_supervised_model(decision_making_model, trainset)

#### Evaluating Model Performance


In [ ]:
from sklearn.metrics import f1_score

def get_f1_score(model, input_testset, device, batch_size=64):
    model.eval()  # Set the model to evaluation mode
    y_true = []
    y_pred = []
    # DataLoader for testing
    test_loader = torch.utils.data.DataLoader(input_testset, batch_size=batch_size, shuffle=False)

    # No gradient computation needed during inference
    with torch.no_grad():
        for data, label in test_loader:
            data = data.to(device)
            label = label.to(device)
            output = model(data)
            # Process output for binary classification
            predicted = (output.squeeze() > 0.5).long()
            y_true.extend(label.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    # Compute and return the F1 score
    score = f1_score(y_true, y_pred, average='binary')
    return score

# Calculate F1 scores for both models
fraud_detection_f1_score = get_f1_score(decision_making_model, testset, device)
random_testset_f1_score = get_f1_score(decision_making_model, random_label_testset, device)
manipulated_testset_f1_score = get_f1_score(decision_making_model, original_label_testset, device)

# Output the results
print("F1 score of the supervised learning model tested on the original data: ", fraud_detection_f1_score)
print("F1 score of the supervised learning model tested on the manipulated data with random label: ", random_testset_f1_score)
print("F1 score of the supervised learning model tested on the manipulated data with original label: ", manipulated_testset_f1_score)